# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Анализировать возможную прибыль и риски будем техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
geo_data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

Данные загружены. Изучим.

### geo_data_0

In [3]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
geo_data_0.head(3)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


* **id** — уникальный идентификатор скважины;
* **f0, f1, f2** — три признака точек;
* **product** — объём запасов в скважине (тыс. баррелей).

In [5]:
geo_data_0.columns

Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

In [6]:
geo_data_0.duplicated().sum()

0

In [7]:
len(geo_data_0.id.unique())

99990

* Скважины повторяются, так как значений в данных 100 000.

* Удалим id скважин, так как не влияет на целевую задачу при построении модели.

In [8]:
geo_data_0 = geo_data_0.drop(columns='id', axis=1)
geo_data_0.head(3)

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647


In [9]:
geo_data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [10]:
geo_data_0[geo_data_0['product'] == 0]

,f0,f1,f2,product
57263,-0.702064,0.375992,0.236572,0.0


### geo_data_1

In [11]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [12]:
geo_data_1.head(3)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


* **id** — уникальный идентификатор скважины;
* **f0, f1, f2** — три признака точек;
* **product** — объём запасов в скважине (тыс. баррелей).

In [13]:
geo_data_1.columns

Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

In [14]:
geo_data_1.duplicated().sum()

0

In [15]:
len(geo_data_1.id.unique())

99996

* Скважины повторяются, так как значений в данных 100 000.

* Удалим id скважин, так как не влияет на целевую задачу при построении модели.

In [16]:
geo_data_1 = geo_data_1.drop(columns='id', axis=1)
geo_data_1.head(3)

,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305


In [17]:
geo_data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [18]:
geo_data_1[geo_data_1['product'] == 0]

,f0,f1,f2,product
11,16.320755,-0.562946,-0.001783,0.0
13,6.695604,-0.749449,-0.007630,0.0
62,21.418478,-5.134490,-0.002836,0.0
63,6.822701,3.104979,-0.000723,0.0
77,6.750150,-11.893512,-0.001601,0.0
...,...,...,...,...
99936,5.085749,-3.980305,0.005063,0.0
99948,8.277805,-9.178818,0.003275,0.0
99956,13.343983,-1.290200,0.005980,0.0
99961,13.854163,-11.528089,-0.005556,0.0


In [19]:
geo_data_1[geo_data_1['product'] == 0].describe()

,f0,f1,f2,product
count,8235.000000,8235.000000,8235.000000,8235.0
mean,8.663769,-3.655963,0.000003,0.0
std,4.994889,5.027089,0.005013,0.0
min,-9.460115,-23.108977,-0.017686,0.0
25%,5.226156,-7.015559,-0.003324,0.0
50%,8.732015,-3.668352,-0.000070,0.0
75%,12.008291,-0.277703,0.003373,0.0
max,25.866134,13.997228,0.020273,0.0


Много нулевых значений. Заполнять медианой не будем. Видимо пустые скважины.

### geo_data_2

In [20]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [21]:
geo_data_2.head(3)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


* **id** — уникальный идентификатор скважины;
* **f0, f1, f2** — три признака точек;
* **product** — объём запасов в скважине (тыс. баррелей).

In [22]:
geo_data_2.columns

Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

In [23]:
geo_data_2.duplicated().sum()

0

In [24]:
len(geo_data_2.id.unique())

99996

* Скважины повторяются, так как значений в данных 100 000.

* Удалим id скважин, так как не влияет на целевую задачу при построении модели.

In [25]:
geo_data_2 = geo_data_2.drop(columns='id', axis=1)
geo_data_2.head(3)

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910


In [26]:
geo_data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [27]:
geo_data_2[geo_data_2['product'] == 0]

,f0,f1,f2,product
68149,-0.865596,-1.615247,-4.126441,0.0


### Вывод

* Пропущенных значений нет;
* id скважин повторяются;
* id для построения модели не важен, столбец удален;
* Дубликатов нет;
* Данные имеют численные признаки;
* geo_data_1 имеет 8235 пустых скважин (8%).

## Обучение и проверка модели

### geo_data_0

In [28]:
geo_data_0.head(3)

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647


In [29]:
features = geo_data_0.drop(['product'], axis=1) 
target = geo_data_0['product'] 
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Применим масштабирование данных.

In [30]:
pd.options.mode.chained_assignment = None

numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

In [31]:
display(features_train.describe())
display(features_valid.describe())

,f0,f1,f2
count,7.500000e+04,7.500000e+04,7.500000e+04
mean,1.794801e-16,-3.105895e-17,-6.015928e-18
std,1.000007e+00,1.000007e+00,1.000007e+00
min,-2.186288e+00,-2.178269e+00,-3.890945e+00
25%,-6.575544e-01,-8.925344e-01,-6.799576e-01
50%,1.878842e-03,-7.407961e-04,4.288679e-03
75%,6.571176e-01,8.928514e-01,6.829752e-01
max,2.139081e+00,2.169192e+00,4.153623e+00


,f0,f1,f2
count,25000.000000,25000.000000,25000.000000
mean,0.013667,0.000629,-0.004025
std,0.999992,1.001848,0.998251
min,-2.064269,-2.096946,-4.491004
25%,-0.645001,-0.897958,-0.692080
50%,0.014689,0.002940,-0.000191
75%,0.673651,0.895259,0.671019
max,2.067368,2.144367,3.849118


In [32]:
model = LinearRegression()
model.fit(features_train, target_train)
predicted_valid_geo_0 = model.predict(features_valid)
mse = mean_squared_error(target_valid, predicted_valid_geo_0)
target_valid_geo_0 = target_valid

print("Linear Regression")
print("RMSE =", mse ** 0.5)
print("Средний запас предсказанного сырья =", predicted_valid_geo_0.mean())
print("R2 на валидационной выборке: ", model.score(features_valid, target_valid)) 
print("MAE = ", mean_absolute_error(target_valid, predicted_valid_geo_0))

predicted_valid = pd.Series(target_train.mean(), index=target_valid.index)
mse = mean_squared_error(target_valid, predicted_valid)

print("")
print("Mean")
print("RMSE =", mse ** 0.5)

predicted_valid = pd.Series(target_train.median(), index=target_valid.index)
print("Median")
print("MAE = ", mean_absolute_error(target_valid, predicted_valid))

Linear Regression
RMSE = 37.5794217150813
Средний запас предсказанного сырья = 92.59256778438035
R2 на валидационной выборке:  0.27994321524487786
MAE =  30.919600777151313

Mean
RMSE = 44.289591053907365
Median
MAE =  37.67178003955267


Запишем в функцию.

In [33]:
def lr_pred(geo_data):

    features = geo_data.drop(['product'], axis=1) 
    target = geo_data['product'] 
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)

    numeric = ['f0', 'f1', 'f2']

    scaler = StandardScaler()
    scaler.fit(features_train[numeric])

    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])

    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid_geo = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid_geo)
    target_valid_geo = target_valid

    print("Linear Regression")
    print("RMSE =", mse ** 0.5)
    print("Средний запас предсказанного сырья =", predicted_valid_geo_0.mean())
    print("R2 на валидационной выборке: ", model.score(features_valid, target_valid)) 
    print("MAE = ", mean_absolute_error(target_valid, predicted_valid_geo_0))

    predicted_valid = pd.Series(target_train.mean(), index=target_valid.index)
    mse = mean_squared_error(target_valid, predicted_valid)

    print("")
    print("Mean")
    print("RMSE =", mse ** 0.5)

    predicted_valid = pd.Series(target_train.median(), index=target_valid.index)
    print("Median")
    print("MAE = ", mean_absolute_error(target_valid, predicted_valid))

    return predicted_valid_geo, target_valid_geo

### geo_data_1

In [34]:
geo_data_1.head(3)

,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305


In [35]:
predicted_valid_geo_1, target_valid_geo_1 = lr_pred(geo_data_1)

Linear Regression
RMSE = 0.8930992867756168
Средний запас предсказанного сырья = 92.59256778438035
R2 на валидационной выборке:  0.9996233978805127
MAE =  46.977255345615276

Mean
RMSE = 46.02144533725471
Median
MAE =  40.26547941945416


### geo_data_2

In [36]:
geo_data_2.head(3)

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910


In [37]:
predicted_valid_geo_2, target_valid_geo_2 = lr_pred(geo_data_2)

Linear Regression
RMSE = 40.02970873393434
Средний запас предсказанного сырья = 92.59256778438035
R2 на валидационной выборке:  0.20524758386040465
MAE =  41.644320201646245

Mean
RMSE = 44.90234968510567
Median
MAE =  37.92103997436729


### Вывод

1. **geo_data_0** Linear Regression:
    * RMSE = 37.5794217150813
    * Средний запас предсказанного сырья = 92.59256778438035
    * R2 на валидационной выборке:  0.27994321524487786
    * MAE =  30.919600777151313


    * Mean: RMSE = 44.289591053907365
    * Median: MAE =  37.67178003955267

2. **geo_data_1** Linear Regression:
    * RMSE = 0.893099286775617
    * Средний запас предсказанного сырья = 68.72854689544602
    * R2 на валидационной выборке:  0.9996233978805127
    * MAE =  0.7187662442124758


    * Mean: RMSE = 46.02144533725462
    * Median: MAE =  40.26547941945416


3. **geo_data_2** Linear Regression:
    * RMSE = 40.02970873393434
    * Средний запас предсказанного сырья = 94.96504596800489
    * R2 на валидационной выборке:  0.20524758386040443
    * MAE =  32.792652105481814


    * Mean: RMSE = 44.90234968510566
    * Median: MAE =  37.92103997436729
    
    

**Вывод:**

* Применили масштабирование к признакам;
* Модели прошли на адекватность предсказаний;
* Лучшая модель в 1 регионе;
* Средние запасы предсказанного сырья:
    * Регион 2: 94.965
    * Регион 0: 92.593
    * Регион 1: 68.729

## Подготовка к расчёту прибыли

In [38]:
PRODUCT_UNIT = 450000 # Доход с каждой единицы продукта
BUDGET = 10000000000 # Бюджет
PRODUCT_POINT = 500 # При разведке региона исследуют 500 точек
PRODUCT_POINT_BEST = 200 # 200 лучших точек для разработки

In [39]:
predicted_product = BUDGET / (PRODUCT_POINT_BEST * PRODUCT_UNIT)
print("Минимальный средний запас сырья в рассчетах: ", predicted_product)
print("Средний запас предсказанного сырья в регионе 0: ", predicted_valid_geo_0.mean())
print("Средний запас предсказанного сырья в регионе 1: ", predicted_valid_geo_1.mean())
print("Средний запас предсказанного сырья в регионе 2: ", predicted_valid_geo_2.mean())

Минимальный средний запас сырья в рассчетах:  111.11111111111111
Средний запас предсказанного сырья в регионе 0:  92.59256778438035
Средний запас предсказанного сырья в регионе 1:  68.728546895446
Средний запас предсказанного сырья в регионе 2:  94.96504596800489


**Вывод:**

    * Минимальный средний запас сырья в рассчетах:  111.11111111111111
    * Средний запас предсказанного сырья в регионе 0:  92.59256778438035
    * Средний запас предсказанного сырья в регионе 1:  68.72854689544602
    * Средний запас предсказанного сырья в регионе 2:  94.96504596800489
    
* Наиболее перспективными по среднему запасу являются регионы 0 и 2.

In [40]:
# функция для рассчета прибыли

def profit(target, predicted, count):
    predicts_sorted = predicted.sort_values(ascending=False)
    selected = target[predicts_sorted.index][:count]
    return PRODUCT_UNIT * selected.sum() - BUDGET

## Расчёт прибыли и рисков 

In [41]:
def geo_pred(predicted_valid_geo, target_valid_geo):
    
    predicted_valid_geo = pd.Series(
        predicted_valid_geo, index=target_valid_geo.index)

    state = np.random.RandomState(12345)

    values = []

    # Bootstrap
    for i in range(1000):
        target_subsample = target_valid_geo.sample(n=PRODUCT_POINT, replace=True, random_state=state)
        predicts_subsample = predicted_valid_geo[target_subsample.index]
        
        values.append(profit(target_subsample, predicts_subsample, PRODUCT_POINT_BEST))

    values = pd.Series(values)

    mean = values.mean()
    print("Средняя прибыль:", mean)

    print('95%-ый доверительный интервал: ({0:.3f}, {1:.3f})'.format(values.quantile(0.025), values.quantile(0.975)))

    # Риск убытков
    count = 0
    for value in values:
        if value < 0:
            count += 1

    loss_risk = (count / len(values) * 100)

    print('Риск убытков {0} %'.format(loss_risk))

    return

### geo_data_0

In [42]:
geo_pred(predicted_valid_geo_0, target_valid_geo_0)

Средняя прибыль: 425938526.910592
95%-ый доверительный интервал: (-102090094.838, 947976353.358)
Риск убытков 6.0 %


### geo_data_1

In [43]:
geo_pred(predicted_valid_geo_1, target_valid_geo_1)

Средняя прибыль: 515222773.44329005
95%-ый доверительный интервал: (68873225.371, 931547591.257)
Риск убытков 1.0 %


### geo_data_2

In [44]:
geo_pred(predicted_valid_geo_2, target_valid_geo_2)

Средняя прибыль: 435008362.782756
95%-ый доверительный интервал: (-128880547.330, 969706954.180)
Риск убытков 6.4 %


### Вывод

**Вывод расчетов geo_data_0:**

    * Средняя прибыль: 425938526.91059244
    * 95%-ый доверительный интервал: (-102090094.838, 947976353.358)
    * Риск убытков 6.0 %
    
    
**Вывод расчетов geo_data_1:**

    * Средняя прибыль: 515222773.4432899
    * 95%-ый доверительный интервал: (68873225.371, 931547591.257)
    * Риск убытков 1.0 %
    
    
**Вывод расчетов geo_data_2:**

    * Средняя прибыль: 435008362.7827556
    * 95%-ый доверительный интервал: (-128880547.330, 969706954.180)
    * Риск убытков 6.4 %
    
    
    

## Общий итог

### Загрузка и подготовка данных:

* Пропущенных значений нет;
* id скважин повторяются;
* id для построения модели не важен, столбец удален;
* Дубликатов нет;
* Данные имеют численные признаки;
* geo_data_1 имеет 8235 пустых скважин (8%).

### Обучение и проверка модели

1. **geo_data_0** Linear Regression:
    * RMSE = 37.5794217150813
    * Средний запас предсказанного сырья = 92.59256778438035
    * R2 на валидационной выборке:  0.27994321524487786
    * MAE =  30.919600777151313


    * Mean: RMSE = 44.289591053907365
    * Median: MAE =  37.67178003955267

2. **geo_data_1** Linear Regression:
    * RMSE = 0.893099286775617
    * Средний запас предсказанного сырья = 68.72854689544602
    * R2 на валидационной выборке:  0.9996233978805127
    * MAE =  0.7187662442124758


    * Mean: RMSE = 46.02144533725462
    * Median: MAE =  40.26547941945416


3. **geo_data_2** Linear Regression:
    * RMSE = 40.02970873393434
    * Средний запас предсказанного сырья = 94.96504596800489
    * R2 на валидационной выборке:  0.20524758386040443
    * MAE =  32.792652105481814


    * Mean: RMSE = 44.90234968510566
    * Median: MAE =  37.92103997436729
    
    
    
    
**Вывод:**

* Применили масштабирование к признакам;
* Модели прошли на адекватность предсказаний;
* Лучшая модель в 1 регионе;
* Средние запасы предсказанного сырья:
    * Регион 2: 94.965
    * Регион 0: 92.593
    * Регион 1: 68.729

### Подготовка к расчёту прибыли

    * Минимальный средний запас сырья в рассчетах:  111.11111111111111
    * Средний запас предсказанного сырья в регионе 0:  92.59256778438035
    * Средний запас предсказанного сырья в регионе 1:  68.72854689544602
    * Средний запас предсказанного сырья в регионе 2:  94.96504596800489
    
* Наиболее перспективными являются регионы 0 и 2.

### Расчёт прибыли и рисков 

**Вывод расчетов geo_data_0:**

    * Средняя прибыль: 425938526.91059244
    * 95%-ый доверительный интервал: (-102090094.838, 947976353.358)
    * Риск убытков 6.0 %
    
    
**Вывод расчетов geo_data_1:**

    * Средняя прибыль: 515222773.4432899
    * 95%-ый доверительный интервал: (68873225.371, 931547591.257)
    * Риск убытков 1.0 %
    
    
**Вывод расчетов geo_data_2:**

    * Средняя прибыль: 435008362.7827556
    * 95%-ый доверительный интервал: (-128880547.330, 969706954.180)
    * Риск убытков 6.4 %

### Общий вывод по проекту

* Несмотря на низкие показатели запасов скважин и наличие большого количества пустых скважин, **регион 1** имеет более высокие показатели по результатам исследования. 

* При бюджете в 10млрд по рассчетам выбор сделан в пользу **регион 1**:

    * Средняя прибыль: 515 222 773
    * 95%-ый доверительный интервал: (68 873 225.371 , 931 547 591.257)
    * Риск убытков 1.0 %
    